In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [4]:
from sklearn.datasets import make_classification

In [7]:
x,y=make_classification(
    n_samples=1000,
    n_features=10,
    n_classes=5,
    n_clusters_per_class=1,
    n_informative=3,
    n_redundant=2,
    random_state=45
)

In [9]:
x[:5]

array([[ 0.32192311,  0.10670382,  0.92972242,  0.54555803,  0.99623275,
        -0.60170514,  0.82063052,  0.11228689, -0.55908945, -0.35509934],
       [ 0.23706224, -0.90252941, -0.86572501, -1.20400881, -1.46952846,
        -0.6871292 , -1.36258409,  1.17215836,  0.62043993,  1.26243837],
       [ 0.20126339,  1.11237341, -0.97099589, -1.54758787, -1.69781061,
        -0.32875808, -1.64413297,  1.68442977,  0.68507459,  1.75308582],
       [ 0.2155875 ,  0.24601413,  0.0716497 , -1.79200723,  2.01809768,
        -0.22331545, -0.04194946, -1.45482707,  0.21840078,  0.4113215 ],
       [-0.74902874,  0.04001198,  0.85232441,  0.88938215, -1.09269298,
        -0.99619293, -0.04866174,  0.28977562,  0.03164596, -1.16367377]])

In [10]:
y[:5]

array([0, 1, 1, 4, 2])

In [12]:
x.shape

(1000, 10)

In [17]:
scaler=StandardScaler()

In [18]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=43)

In [19]:
x_train.shape

(800, 10)

In [20]:
x_test.shape

(200, 10)

In [22]:
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

(800, 10)

In [26]:
regressor=LogisticRegression(multi_class="ovr")

In [25]:
cv=StratifiedKFold()

In [27]:
regressor.fit(x_train,y_train)

LogisticRegression(multi_class='ovr')

In [29]:
y_pred=regressor.predict(x_test)

In [30]:
regressor.get_params

<bound method BaseEstimator.get_params of LogisticRegression(multi_class='ovr')>

In [32]:
m1=accuracy_score(y_test,y_pred)
m2=confusion_matrix(y_test,y_pred)
m3=classification_report(y_test,y_pred)


print(f"m1 : {m1}")
print(f"m2 : {m2}")
print(f"m3 : {m3}")

m1 : 0.74
m2 : [[38  0  1  8  2]
 [ 0 27  3  1  6]
 [ 0  0 27  6  2]
 [ 0  1  6 27  7]
 [ 1  0  1  7 29]]
m3 :               precision    recall  f1-score   support

           0       0.97      0.78      0.86        49
           1       0.96      0.73      0.83        37
           2       0.71      0.77      0.74        35
           3       0.55      0.66      0.60        41
           4       0.63      0.76      0.69        38

    accuracy                           0.74       200
   macro avg       0.77      0.74      0.74       200
weighted avg       0.77      0.74      0.75       200



In [53]:
# Common parameters
C = [0.1, 1.0, 1.5, 2.0]
fit_intercept = [True]
class_weight = [None]
random_state = [None]
max_iter = [100, 200, 300, 400]
multi_class = ['auto', 'ovr', 'multinomial']
verbose = [0]
n_jobs = [-1]
l1_ratio = [0.9]  # Only used when penalty='elasticnet' and solver='saga'

# Build compatible parameter grids
params = [
    # Group 1: Solvers supporting only l2 and none penalties
    {
        'penalty': ['l2', 'none'],
        'C': C,
        'fit_intercept': fit_intercept,
        'class_weight': class_weight,
        'random_state': random_state,
        'solver': ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag'],
        'max_iter': max_iter,
        'multi_class': multi_class,
        'verbose': verbose,
        'n_jobs': n_jobs
    },
    # Group 2: liblinear supports l1 and l2 (optional)
    {
        'penalty': ['l1', 'l2'],
        'C': C,
        'fit_intercept': fit_intercept,
        'class_weight': class_weight,
        'random_state': random_state,
        'solver': ['saga'],
        'max_iter': max_iter,
        'multi_class': multi_class,
        'verbose': verbose,
        'n_jobs': n_jobs
    },
    # Group 3: elasticnet with saga only
    {
        'penalty': ['elasticnet'],
        'C': C,
        'fit_intercept': fit_intercept,
        'class_weight': class_weight,
        'random_state': random_state,
        'solver': ['saga'],
        'max_iter': max_iter,
        'multi_class': multi_class,
        'verbose': verbose,
        'n_jobs': n_jobs,
        'l1_ratio': l1_ratio  # l1_ratio only for elasticnet
    }
]


In [54]:
gridcv=GridSearchCV(
    estimator=regressor,
    param_grid=params,
    cv=cv,
    scoring="accuracy"
)

In [55]:
gridcv.fit(x_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(multi_class='ovr'),
             param_grid=[{'C': [0.1, 1.0, 1.5, 2.0], 'class_weight': [None],
                          'fit_intercept': [True],
                          'max_iter': [100, 200, 300, 400],
                          'multi_class': ['auto', 'ovr', 'multinomial'],
                          'n_jobs': [-1], 'penalty': ['l2', 'none'],
                          'random_state': [None],
                          'solver': ['lbf...
                          'n_jobs': [-1], 'penalty': ['l1', 'l2'],
                          'random_state': [None], 'solver': ['saga'],
                          'verbose': [0]},
                         {'C': [0.1, 1.0, 1.5, 2.0], 'class_weight': [None],
                          'fit_intercept': [True], 'l1_ratio': [0.9],
                          'max_iter': [100, 200, 300, 400],
                          'multi_class': ['auto', 'ovr', 'multinomial'],
                          'n_jobs': [-1], 'penalty': ['elasticnet'],
                          'random_state': [None], 'solver': ['saga'],
                          'verbose': [0]}],
             scoring='accuracy')

In [56]:
gridcv.best_estimator_

LogisticRegression(C=0.1, l1_ratio=0.9, multi_class='ovr', n_jobs=-1,
                   penalty='elasticnet', solver='saga')

In [57]:
gridcv.best_params_

{'C': 0.1,
 'class_weight': None,
 'fit_intercept': True,
 'l1_ratio': 0.9,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': -1,
 'penalty': 'elasticnet',
 'random_state': None,
 'solver': 'saga',
 'verbose': 0}

In [58]:
gridcv.best_score_

np.float64(0.7925)

In [59]:
y_pred1=gridcv.predict(x_test)

In [60]:
m1=accuracy_score(y_test,y_pred1)
m2=confusion_matrix(y_test,y_pred1)
m3=classification_report(y_test,y_pred1)


print(f"m1 : {m1}")
print(f"m2 : {m2}")
print(f"m3 : {m3}")

m1 : 0.73
m2 : [[36  0  1 10  2]
 [ 0 27  3  1  6]
 [ 0  1 28  4  2]
 [ 0  1  6 27  7]
 [ 1  0  1  8 28]]
m3 :               precision    recall  f1-score   support

           0       0.97      0.73      0.84        49
           1       0.93      0.73      0.82        37
           2       0.72      0.80      0.76        35
           3       0.54      0.66      0.59        41
           4       0.62      0.74      0.67        38

    accuracy                           0.73       200
   macro avg       0.76      0.73      0.74       200
weighted avg       0.77      0.73      0.74       200

